In [1]:
"""
Created on Tue May  7 16:50:14 2019
For filenames in the folder, load in ground truth1|2 json
run lt_evaluate on all ROilist files, export as a pandas table
@author: lan
"""

'\nCreated on Tue May  7 16:50:14 2019\nFor filenames in the folder, load in ground truth1|2 json\nrun lt_evaluate on all ROilist files, export as a pandas table\n@author: lan\n'

In [1]:
import os
import json
import numpy as np
from neurofinder import centers, shapes
import pandas as pd
import re
from regional import one, many

In [2]:
def load(file):
    """
    Load neuronal regions from a file or string.
    """
    if os.path.isfile(file):
        with open(file, 'r') as f:
            values = json.load(f)
    else:
        values = json.loads(file)
    
    if type(values) is dict:
        values=[values]
    if len(values)>0 and 'coordinates' in values[0].keys():
        return many([v['coordinates'] for v in values])
    else:
        return 0

In [3]:
def lt_evaluate(file1,file2,threshold=5):
    a = load(file1)
    b = load(file2)
    if a!=0 and b!=0:
        recall, precision = centers(a, b, threshold=threshold)
        inclusion, exclusion = shapes(a, b, threshold=threshold)

        if recall == 0 and precision == 0:
            combined = 0
        else:
            combined = 2 * (recall * precision) / (recall + precision)
    
        result = {'combined': round(combined, 4), 'inclusion': round(inclusion, 4), 'precision': round(precision, 4), 'recall': round(recall, 4), 'exclusion': round(exclusion, 4)}
        return(result)
    else:
        return {'combined': np.nan, 'inclusion': np.nan, 'precision': np.nan, 'recall': np.nan, 'exclusion': np.nan}

In [61]:
figsize=[256,256]
pathname="/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Older/"
pathname_labels="/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Older/2pROI-sum/"
pathname_lssc="/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Older/LSSC_results/2p_ROI/"

In [8]:
videos = []
for r, d, f in os.walk(pathname): # r=root, d=directories, f = files
    for file in f:
        if '.tif' in file: #extract file names
            videos.append(os.path.join(file[0:-4]))

In [9]:
videos

['File1_CPn_l5_gcamp6s_lan',
 'File2_CSt_l5_gcamp6s_lan',
 'File3_kf139_s',
 'File4_kf158_s',
 'File5_l23_gcamp6s_lan',
 'File6_som_l5_gcamp6s_alka']

In [14]:
for v in videos:
    print(v[6:]+'_ROIlist.json')

CPn_l5_gcamp6s_lan_ROIlist.json
CSt_l5_gcamp6s_lan_ROIlist.json
kf139_s_ROIlist.json
kf158_s_ROIlist.json
l23_gcamp6s_lan_ROIlist.json
som_l5_gcamp6s_alka_ROIlist.json


In [22]:
for v in videos:
    # load in ground truth
    GT_int=pathname+'GroundTruth/'+v+'_all_intersect.json'
    GT_uni=pathname+'GroundTruth/'+v+'_all_uniion.json'
    
    print(GT_int)
    print(GT_uni)

/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File1_CPn_l5_gcamp6s_lan_all_intersect.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File1_CPn_l5_gcamp6s_lan_all_uniion.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File2_CSt_l5_gcamp6s_lan_all_intersect.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File2_CSt_l5_gcamp6s_lan_all_uniion.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File3_kf139_s_all_intersect.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File3_kf139_s_all_uniion.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File4_kf158_s_all_intersect.json
/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Files for GUI analysis/GroundTruth/File4_kf158_s_

In [67]:
for v in videos:
    # load in ground truth
    GT_int=pathname+'GroundTruth/'+v+'_all_intersect.json'
    GT_uni=pathname+'GroundTruth/'+v+'_all_union.json'
    
    # declare result matrix
    res_intersect=[]
    res_union=[]
    # search through all files generated by the algorithm
    for r,d,f in os.walk(pathname_labels):
        for file in f:
            name = v[6:]+'_ROIlist.json'
            if (name in file):
                source=(os.path.split(r)[1])
                
                currentfilename=os.path.join(r,file)
                #print(currentfilename)
                result=lt_evaluate(GT_int,currentfilename,10)
                result.update(dict(zip(['source'],[source])))
                res_intersect.append(result)
                
                result=lt_evaluate(GT_uni,currentfilename,10)
                result.update(dict(zip(['source'],[source])))
                res_union.append(result)
    print(v)
    #print("Comparison to intersection GT")
    #print(res_intersect)
    
    df=pd.DataFrame(res_intersect)
    df=df[['source','combined','precision','recall','exclusion','inclusion']]
    print(df)
    #print("Compariosn to union GT")
    #print(res_union)
                

File1_CPn_l5_gcamp6s_lan
  source  combined  precision  recall  exclusion  inclusion
0   alka    0.7500     0.9231  0.6316     0.9790     0.6662
1    dan    0.7708     0.9487  0.6491     0.9966     0.3142
2  Katie    0.7500     0.9231  0.6316     0.9904     0.4252
3    lan    0.8468     0.8704  0.8246     0.9279     0.6316
File2_CSt_l5_gcamp6s_lan
  source  combined  precision  recall  exclusion  inclusion
0   alka    0.6341     0.9630  0.4727     0.9587     0.7379
1    dan    0.6265     0.9286  0.4727     0.9801     0.5396
2  Katie    0.6744     0.9355  0.5273     0.9830     0.4811
3    lan    0.7835     0.9048  0.6909     0.9397     0.7335
File3_kf139_s
  source  combined  precision  recall  exclusion  inclusion
0   alka    0.8649     1.0000  0.7619     0.9603     0.7848
1    dan    0.7895     0.8824  0.7143     0.9854     0.4614
2  Katie    0.8333     1.0000  0.7143     0.9673     0.5255
3    lan    0.8421     0.9412  0.7619     0.9327     0.8935
File4_kf158_s
  source  combined  pr

In [64]:
pathname_lssc="/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Older/LSSC_results/2p_ROI/"

for v in videos:
    # load in ground truth
    GT_int=pathname+'GroundTruth/'+v+'_all_intersect.json'
    GT_uni=pathname+'GroundTruth/'+v+'_all_union.json'
    
    # declare result matrix
    res_intersect=[]
    # search through all files generated by the algorithm
    for r,d,f in os.walk(pathname_lssc):
        name = v[6:]
        print(name)
        for file in f:
            if (name in file) and ('json' in file):
                currentfilename=os.path.join(r,file)
                
                # extract processing parameters
                para=re.findall('preproc\_(.*?)\_normal\_(.*?)\_pca\_(.*?)\_ntrial\_(.*?)\_neig\_(.*?)\_nclust\_(.*?)\_out',\
                           currentfilename)
                para=[int(x) for x in para[0]]               
                parameters=dict(zip(['preproc','normal','pca','ntrial','neig','nclust'],para))
                
                result=lt_evaluate(GT_int,currentfilename,10)
                result.update(parameters)
                res_intersect.append(result)
                
                #result=lt_evaluate(GT_uni,currentfilename,15)
                #result.update(parameters)
                #res_union.append(result)
    
    df=pd.DataFrame(res_intersect)
    df=df[['preproc','normal','pca','ntrial','neig','nclust','combined','precision','recall','exclusion','inclusion']]
    df.to_csv(pathname+'/Results//'+v+'_GT_int_result.csv')

    #df=pd.DataFrame(res_union)
    #df=df[['preproc','normal','pca','ntrial','neig','nclust','combined','precision','recall','exclusion','inclusion']]   
    #df.to_csv(pathname+'Results\\'+v+'_GT_uni_result.csv')

CPn_l5_gcamp6s_lan
CSt_l5_gcamp6s_lan
kf139_s
kf158_s
l23_gcamp6s_lan
som_l5_gcamp6s_alka


In [68]:
pathname_suite2p = "/Users/mishne/Dropbox/Yale/data/Imaging Analysis Group/Older/Suite2p_dan/"

for v in videos:
    # load in ground truth
    GT_int=pathname+'GroundTruth/'+v+'_all_intersect.json'
    GT_uni=pathname+'GroundTruth/'+v+'_all_union.json'
    
    # declare result matrix
    res_intersect=[]
    res_union=[]
    # search through all files generated by the algorithm
    for r,d,f in os.walk(pathname_suite2p):
        for file in f:
            name = v[6:]+'_suite2p_ROIlist.json'
            if (name in file):
                source="suite2p"
                
                currentfilename=os.path.join(r,file)
                #print(currentfilename)
                result=lt_evaluate(GT_int,currentfilename,10)
                result.update(dict(zip(['source'],[source])))
                res_intersect.append(result)
                
                result=lt_evaluate(GT_uni,currentfilename,10)
                result.update(dict(zip(['source'],[source])))
                res_union.append(result)
    print(v)
    #print("Comparison to intersection GT")
    #print(res_intersect)
    
    df=pd.DataFrame(res_intersect)
    df=df[['source','combined','precision','recall','exclusion','inclusion']]
    print(df)
    #print("Compariosn to union GT")
    #print(res_union)

File1_CPn_l5_gcamp6s_lan
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.6882     0.8889  0.5614     0.8651     0.9594
File2_CSt_l5_gcamp6s_lan
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.3889     0.8235  0.2545     0.8131     0.9645
File3_kf139_s
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.1667        0.2  0.1429     0.2585     0.9963
File4_kf158_s
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.8188     0.8309  0.8071     0.8329     0.9129
File5_l23_gcamp6s_lan
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.6923     0.9545  0.5431     0.9002     0.8892
File6_som_l5_gcamp6s_alka
    source  combined  precision  recall  exclusion  inclusion
0  suite2p    0.2286        1.0   0.129     0.8081     0.8901
